In [ ]:
import sys
import os

# Add the project root directory to sys.path
project_root = os.path.abspath(".") 
if project_root not in sys.path:
    sys.path.append(project_root)
print(sys.path[-1])


# reward function 

In [3]:
# Define a custom reward function
def coord_reward_func(state):
    # Add artificial delay to simulate expensive computation
    # time.sleep(0.1)  
    
    # reward1 = sum(1 for coord in state if coord == 5) + 0.001 # args.min_reward
    reward1 = sum(1 for coord in state if coord == 6) 
    reward2 = sum(2 for coord in state if coord == 8) 
    return reward1 + reward2


In [ ]:

test_weights = [-60, -150, 0, -109, 65, -66, -145, -41, 58] # -41(41), 58 
# test_weights = [44, -24, -26, 23, 55, 25, -51, 1, -3]
test_weights = [154, -200, 82, -33, 90, -90, -82, -13, 30] 
reward = somitogenesis_reward_func(test_weights, plot=True)
print(f"Reward for somitogenesis: {reward}")


In [ ]:
# test_weights = [-5, 2, 3, 1, 2, -1, 2, 1, -3] 
# test_weights = [0, -10, 0, 4, 6, -10, -10, 0, 0]  # w12=-5, w22=..., w23=-10, w31=-10 
test_weights = [154, -200, 82, -33, 90, -90, -82, -13, 30] 
reward = oscillator_reward_func(test_weights, plot=True)
print(f"Reward for oscillator: {reward}")

# Analysis

In [ ]:
title = "Loss and Z for the ({}) Model".format(args.method)
plot_loss_curve(losses, zs=zs, title=title)
print("The final Z (partition function) estimate is {:.2f}".format(zs[-1]))

# print(agent.state_counts)
# print(agent.state_returns)

In [ ]:
agent.env.print_actions()

In [ ]:
print("\n")
print("-" * 60)
print(f"Number of unique terminal states found: {len(agent.ep_last_state_counts)}")

print("-" * 30)
print("Top 12 by Avg avg trajectory rewards:")
print("-" * 30)

# Calculate average of average trajectory rewards for each state
state_avg_rewards = {}
for state, trajectories in agent.ep_last_state_trajectories.items():
    traj_avgs = []
    for traj in trajectories:
        rewards = [r[0] for r in traj['rewards']]
        traj_avgs.append(sum(rewards) / len(rewards))
    state_avg_rewards[state] = sum(traj_avgs) / len(traj_avgs)

# Sort states by state_avg_rewards
top_reward_states = sorted(
    state_avg_rewards.items(),
    key=lambda x: x[1],
    reverse=True
)[:12]

for state, avg_reward in top_reward_states:
    trajectories = agent.ep_last_state_trajectories[state]
    count = agent.ep_last_state_counts[state]
    terminal_reward = trajectories[0]['rewards'][-1][0]
    print(f"State: {state}, Count: {count}, Terminal reward: {terminal_reward:.3f}, Avg avg trajectory reward: {avg_reward:.3f}")
    
    # Print all trajectories for this state
    print("All trajectories:")
    for i, traj in enumerate(trajectories):
        rewards = [r[0] for r in traj['rewards']]
        traj_avg = sum(rewards) / len(rewards)
        print(f"\nTrajectory {i+1}:")
        print(f"Rewards: {[f'{r:.3f}' for r in rewards]}, Average: {traj_avg:.3f}")
        print("States along trajectory:")
        for j, state_encoding in enumerate(traj['states']):
            state = agent.encoding_to_state(state_encoding)
            if agent.env.enable_time:
                state = tuple(state[1:])  # Remove time component
            print(f"Step {j}: {state}")
    print("")

print("-" * 30)
print("Top 12 by visit count:")
print("-" * 30)

top_count_states = sorted(
    agent.ep_last_state_counts.items(),
    key=lambda x: x[1],
    reverse=True
)[:12]

for state, count in top_count_states:
    trajectories = agent.ep_last_state_trajectories[state]
    terminal_reward = trajectories[0]['rewards'][-1][0]
    print(f"State: {state}, Count: {count}, Terminal reward: {terminal_reward:.3f}, Avg avg trajectory reward: {state_avg_rewards[state]:.3f}")
    
    # Print top 3 trajectories by average reward
    sorted_trajectories = sorted(
        trajectories,
        key=lambda t: sum(r[0] for r in t['rewards']) / len(t['rewards']),
        reverse=True
    )[:3]
    for traj in sorted_trajectories:
        rewards = [r[0] for r in traj['rewards']]
        traj_avg = sum(rewards) / len(rewards)
        print(f"Trajectory rewards: {[f'{r:.3f}' for r in rewards]}, Average: {traj_avg:.3f}")
    print("")
print("\n")


# testing

In [ ]:

"""Test case: Mixed positive/negative actions"""

args_mixed = argparse.Namespace(
    min_reward=0.001,
    custom_reward_fn=coord_reward_func,
    n_steps=5,
    n_dims=2,
    actions_per_dim=[1, -1, 2, -2],  # Mixed positive and negative actions
    grid_bound=5,
    enable_time=True
)
env_mixed = GridEnv(args_mixed)
env_mixed.print_actions()
env_mixed.reset()

mixed_test_state = (17, [-5, -5])
print("forward: ", env_mixed.get_forward_mask(mixed_test_state))
print("backward: ", env_mixed.get_backward_mask(mixed_test_state))
print("step: ", env_mixed._step)
print()  # Add blank line between iterations



In [ ]:
import time

# Function that simulates an expensive computation
def expensive_computation(x):
    time.sleep(0.1)  # Simulate work
    return x * 2

# Test without cache
print("Without cache:")
start_time = time.time()
for i in range(10):
    # Compute same values multiple times
    result = expensive_computation(i % 3)  # Only compute 0,1,2 repeatedly
    print(f"Computing {i % 3} -> {result}")
no_cache_time = time.time() - start_time
print(f"Time without cache: {no_cache_time:.2f}s\n")

# Test with cache
print("With cache:")
cache = LRUCache(max_size=3)
start_time = time.time()
for i in range(10):
    key = i % 3
    if key in cache:
        result = cache[key]
        print(f"Cache hit for {key} -> {result}")
    else:
        result = expensive_computation(key)
        cache[key] = result
        print(f"Cache miss for {key} -> {result}")
cache_time = time.time() - start_time
print(f"Time with cache: {cache_time:.2f}s")

print(f"\nCache speedup: {no_cache_time/cache_time:.1f}x faster")
